In [1]:
from glob import glob
import codecs
from datetime import datetime
import numpy
import pandas

<h3>資料樣例</h3>
<pre>
Date	Open	High	Low	Close	AdjClose	Volume
Dec 30, 2020	12,860.75	12,909.75	12,820.50	12,841.50	12,841.50	334,304
Dec 29, 2020	12,845.25	12,918.25	12,808.25	12,841.00	12,841.00	393,995
Dec 28, 2020	12,676.25	12,855.50	12,655.25	12,832.75	12,832.75	346,691
Dec 24, 2020	-	-	-	-	-	-
</pre>

<h3>環境設置</h3>

In [2]:
data_path = 'D:/Projects/DB/data'
file_list = glob('%s/NQF-*.txt' % data_path)
file_list

['D:/Projects/DB/data\\NQF-2017.txt',
 'D:/Projects/DB/data\\NQF-2018.txt',
 'D:/Projects/DB/data\\NQF-2019.txt',
 'D:/Projects/DB/data\\NQF-2020.txt']

In [3]:
datatable = 'StockInfo'

<h3>工具函數</h3>
<h3>#在變數前面加_代表是一個暫存的變數檔案(業界不成文習慣)</h3>

In [4]:
def import_data(stockID, file_name):
    _dataset = pandas.read_csv(
        file_name, delimiter='\t', parse_dates=[0], infer_datetime_format=True,
        header=0, names=['StockDate', 'StockOpen', 'StockHigh', 
                         'StockLow', 'StockClose', 'StockAdjClose', 'StockVol'],
    )
    #delimiter='\t'：欄位間隔為tab鍵
    #parse_dates=[0]：定義日期欄位在哪裡
    #infer_datetime_format=True：自動判斷資料格式
    #header=0：不要欄位名稱
    #names=[]：重新定義欄位名稱
    
    _dataset['StockID'] = stockID
    _dataset = _dataset.sort_values(by=['StockDate'])
    
    _dataset.reset_index(drop=True, inplace=True) #不要有index的效果
    #drop=True：不管前面有沒有設置好的index都刪掉
    #inplace=True：取代原本的內部資料結構
    
    _dataset = _dataset.loc[_dataset['StockVol'] != '-']
    
    _number_fields = ['StockOpen', 'StockHigh', 'StockLow', 'StockClose', 
                      'StockAdjClose', 'StockVol']
    #定義想要變成數值欄位的欄位名稱
    
    _dataset[_number_fields] = _dataset[_number_fields].apply(
        lambda x: pandas.to_numeric(x.astype(str).str.replace(',', ''), errors='coerce')
    )
    #先把x轉成字串，呼叫字串將,取代成空值，最後再轉為數值(numeric)型態
    #apply()：執行後方的短函數
    #x.astype(str).str：將x轉成字串並呼叫字串
    #replace(',', '')：將,(千分位的,)替換成空值
    
    _dataset['StockVol'] = _dataset['StockVol'] / 1000.0
            
    return _dataset[['StockID', 'StockDate', 
                    'StockOpen', 'StockHigh', 'StockLow', 'StockClose', 
                    'StockAdjClose', 'StockVol']]
    #改變欄位顯示順序

<h3>載入檔案</h3>

In [5]:
_dataset = import_data('NQF', file_list[0])

In [6]:
_dataset.tail()

,StockID,StockDate,StockOpen,StockHigh,StockLow,StockClose,StockAdjClose,StockVol
246,NQF,2017-12-22,6489.75,6501.75,6467.50,6483.75,6483.75,130.810
247,NQF,2017-12-26,6482.50,6492.00,6432.25,6460.00,6460.00,135.130
248,NQF,2017-12-27,6461.25,6475.50,6442.00,6454.00,6454.00,158.481
249,NQF,2017-12-28,6452.25,6477.50,6442.25,6446.25,6446.25,138.254
250,NQF,2017-12-29,6444.50,6467.75,6383.25,6408.75,6408.75,210.114


<h3>建立表格 SQL 命令稿</h3>

In [7]:
_create_table_sql = pandas.io.sql.get_schema(_dataset, datatable).split('\n') #把資料轉成SQL語句，並用split('\n')分隔成陣列形式
_create_table_sql

['CREATE TABLE "StockInfo" (',
 '"StockID" TEXT,',
 '  "StockDate" TIMESTAMP,',
 '  "StockOpen" REAL,',
 '  "StockHigh" REAL,',
 '  "StockLow" REAL,',
 '  "StockClose" REAL,',
 '  "StockAdjClose" REAL,',
 '  "StockVol" REAL',
 ')']

<h3>新增筆錄至表格內 SQL 命令稿</h3>

In [8]:
_sql = 'INSERT INTO %s ' % datatable
_sql = _sql + '(%s, %s, %s, %s, %s, %s, %s, %s) ' % tuple(_dataset.columns) #格式化的表示法要使用tuple

_insert_table_sql_list = [] #產生一個空陣列

for i in range(_dataset.shape[0]): #看有多少筆資料就跑幾次迴圈
    _row = _dataset.iloc[i]
    _insert_table_sql = _sql + "VALUES('%s','%s',%s,%s,%s,%s,%s,%s)" % (
        _row['StockID'], _row['StockDate'].strftime('%Y-%m-%d'), #strftime：字串轉日期格式
        _row['StockOpen'], _row['StockHigh'], _row['StockLow'], _row['StockClose'],
        _row['StockAdjClose'], _row['StockVol']
    )    
    
    _insert_table_sql_list.append(_insert_table_sql) #把每一筆資料加到陣列

_insert_table_sql_list[0]

"INSERT INTO StockInfo (StockID, StockDate, StockOpen, StockHigh, StockLow, StockClose, StockAdjClose, StockVol) VALUES('NQF','2017-01-03',4876.75,4927.5,4876.75,4906.25,4906.25,215.983)"

In [9]:
_sql

'INSERT INTO StockInfo (StockID, StockDate, StockOpen, StockHigh, StockLow, StockClose, StockAdjClose, StockVol) '

In [10]:
_insert_table_sql = _sql + "VALUES('%s','%s',%s,%s,%s,%s,%s,%s)" % (
        'NQF', '2020-12-24', -100.0, -100.0, -100.0, -100.0, -100.0, 100.0
    )  

_insert_table_sql

"INSERT INTO StockInfo (StockID, StockDate, StockOpen, StockHigh, StockLow, StockClose, StockAdjClose, StockVol) VALUES('NQF','2020-12-24',-100.0,-100.0,-100.0,-100.0,-100.0,100.0)"

<h3>異動筆錄至表格內 SQL 命令稿</h3>

In [11]:
_update_table_sql = "UPDATE %s SET %s=%s WHERE StockID='%s' AND StockDate='%s'" % (
    datatable, 'StockVol', 0.0, 'NQF', '2020-12-24')
#WHERE：給定update的條件

_update_table_sql

"UPDATE StockInfo SET StockVol=0.0 WHERE StockID='NQF' AND StockDate='2020-12-24'"

<h3>刪除筆錄至表格內 SQL 命令稿</h3>

In [12]:
_delete_table_sql = "DELETE FROM %s WHERE StockID='%s' AND StockDate='%s'" % (
    datatable, 'NQF', '2020-12-24')

_delete_table_sql

"DELETE FROM StockInfo WHERE StockID='NQF' AND StockDate='2020-12-24'"